In [ ]:
from bokeh.charts import Bar, output_notebook, show
from bokeh.io import gridplot

In [ ]:
inputFile = 'speciesDistribution.txt'
handle = open(inputFile, 'r')

sites = {}
taxa = []

for line in handle.readlines():
    line = line.strip()
    if line.startswith('Site'):
        siteName = line[6:]
        sites[siteName] = {}
    else:
        taxonID, count = line.split('\t')
        if taxonID not in taxa:
            taxa.append(taxonID)
        sites[siteName][taxonID] = int(count)


In [ ]:
siteNames = sites.keys()
siteNames = sorted(siteNames)
taxa.sort()

In [ ]:
for site in siteNames:
    for taxonID in taxa:
        if taxonID not in sites[site]:
            sites[site][taxonID] = 0


In [ ]:
output_notebook()

In [ ]:
plots = []
for site in siteNames:
    data = {}
    data[site] = []
    data['taxa'] = taxa
    for taxon in taxa:
        data[site].append(sites[site][taxon])
    plot = Bar(data, 'taxa', values=site, color='taxa', title=site, xlabel='taxon', ylabel='abundance', tools='')
    plots.append(plot)

In [ ]:
layout = gridplot([[plots[0],plots[1]],\
                   [plots[2],plots[3]],\
                   [plots[4],plots[5]],\
                   [plots[6]]])
show(layout)